In [20]:
import os
import csv
import functools
root='./data/4/0808(1000組)/'
all=os.listdir(root)
n=4
tot=0
pol=[]
for i in all:
    if(i=='text.txt'):
        continue
    with open(root+i,'r') as csv1:
        rd=csv1.readlines()
        tot+=len(rd)/2
        pol+=[[i.replace('.csv',''),len(rd)/2]]
print(tot)

10000.0


In [21]:
cnt=[[0for i in range(4)]for j in range(4)]
for i in pol:
    pl=[]
    for j in (i[0][1:-1]).split(','):
        pl+=[int(j)]
    for j in range(n):
        cnt[j][pl[j]]+=i[1]


In [22]:
for i in cnt:
    for j in i:
        print(round(j*100/tot,2),end='  ')
    print()

5.03  32.38  30.88  31.71  
31.02  5.01  32.97  31.0  
29.94  31.93  4.95  33.18  
32.58  30.4  31.95  5.07  


In [23]:
def per(cur=[]):
    n=4
    if(len(cur)==n):
        return [cur]
    ret=[]
    for i in range(n):
        if(not i in cur):
            ret+=per(cur+[i])
    return ret
p=per()
print(len(p))

24


In [24]:
cnt=dict()
for f in all:
    with open(root+f,'r') as csv1:
        rd=csv.reader(csv1)
        for i in rd:
            if(i==[]):
                continue
            s=[]
            for j in i:
                s+=[float(j)]
            ss=sorted(s)
            cur=[]
            for j in p:
                if(s==[ss[j[0]],ss[j[1]],ss[j[2]],ss[j[3]]]):
                    cur=j
                    break
           #cur[i]=j  ==>  unsorted[i]<->sorted[j]
            p0=[]
            for j in f.replace('.csv','')[1:-1].split(','):
                p0+=[int(j)]
            p1=[-1 for j in range(n)]
            for j in range(n):
                p1[cur[j]]=cur[p0[j]]
            if(cnt.get(str(p1))==None):
                cnt[str(p1)]=0
            cnt[str(p1)]+=1

In [18]:
def cmp(a,b):
    if(a[1]<b[1]):
        return 1
    return -1
c1=[]
for i in cnt.keys():
    c1+=[[i,cnt[i]]]
c1.sort(key=functools.cmp_to_key(cmp))
for i in c1:
    #print(i[0],':',round(i[1]*100/tot,3))
    pass

In [25]:
W=dict()
def w(pi,p,pol):#pi=要計算勝率的參賽者，p為一陣列，p[i]=參賽者Pi的命中率，pol為策略集，pol[i]為Pi的策略，若pol[i]=i則代表該參賽者策略為放棄
  n=len(p)

  if(W.get(str(p))!=None and W[str(p)].get(str(pol))!=None and W[str(p)][str(pol)].get(pi)!=None):
    return W[str(p)][str(pol)][pi]


  if(n==1):#若場上剩下參賽者一人，則該參賽者勝利
    return 1
  if(n==3):
    return w3(pi,p,pol)
  #檢查是否所有人策略都為放棄
  ok=False
  for i in range(n):
    if(pol[i]!=i):
      ok=True
      break
  if(not ok):
    return 0


  r=1#紀錄「到目前為止，前一名參賽者都沒有射中目標的機率」
  ret=0#記錄勝率
  per=[i for i in range(n)]
  
  for j in range(n):#一般式中的sigma
    if(pol[j]==j):#若該參賽者放棄射擊則直接跳過
      continue
    if(pol[j]!=pi):#若該參賽者策略不為「射擊Pi」，則Pi勝率可累加上退化後賽局中的勝率
      p2=p[j+1:]+p[:j+1]#將命中率陣列以退化後的射擊順序重排
      
      #刪除遭射中的參賽者
      d=(pol[j]+n-j-1)%n
      del p2[d]

      #Pi在退化後賽局的位置
      pi2=(pi+n-j-1)%n

      if(pi2>d):
        pi2-=1

      pl2=ABL(p2)#尋找退化後賽局的平衡策略
      ret+=r*p[j]*w(pi2,p2,pl2)#將退化後賽局的勝率累加

    r*=(1-p[j])
  ret/=(1-r)
  if(W.get(str(p))==None):
    W[str(p)]=dict()
  if(W[str(p)].get(str(pol))==None):
    W[str(p)][str(pol)]=dict()
  W[str(p)][str(pol)][pi]=ret
  return ret
def w3(pi,p,pol):#s是命中率陣列，此函數可求出三人命中率分別為(s[0],s[1],s[2])，策略集為pol下Pi的勝率
  if(pol[0]==0 and pol[1]==1 and pol[2]==2):
    return 0
  ret=0
  r=1
  for i in range(3):
    i1=(i)
    if(pol[i1]==i1):
      continue
    if(i1==pi):
      k=3-pi-pol[pi]
      ret+=r*p[pi]*(1-p[k])*(p[pi])/(1-(1-p[pi])*(1-p[k]))
    elif (pol[i1]!=pi):
      ret+=r*(p[i1])*(p[pi])/(1-(1-p[pi])*(1-p[i1]))
    r*=(1-p[i1])
  ret/=(1-r)
  return ret
def ABL(s,cur=[]):
  pi=len(cur)
  if(len(cur)==len(s)):
    return cur
  abl=[]
  for pli in range(len(s)):
    abl2=ABL(s,cur+[pli])
    if (abl==[]):
      abl=abl2
    elif (w(pi,s,abl2)>w(pi,s,abl)):
      abl=abl2
  return abl

In [41]:
import time
import random
cnt_w=[0 for i in range(n)]
pl=[3,0,3,2]
t0=1000
t=time.time()
for i0 in range(t0):
    s=[random.random() for i in range(n)]
    s.sort()
    for p in per():#p[i]=j ==> unsorted[j]<->sorted[i]
        s1=[-1 for i in range(n)]
        for i in range(4):
            s1[p[i]]=s[i]
        p1=[-1 for i in range(n)]
        for i in range(4):
            p1[p[i]]=p[pl[i]]
        #print(s)
        #print(s1)
        #print(p1)
        #print()
        wp=[w(pi,s1,p1) for pi in range(n)]
        for i in range(4):
            cnt_w[i]+=wp[p[i]]
for i in cnt_w:
    print(round(i/(24*t0),2),end=' ')
print('\n',time.time()-t)

0.21 0.35 0.17 0.27 
 101.92400002479553
